### **Move to Reference Pose with Intermediate Direction**
* This Colab notebook is created by Sarah 97963A (Discord: `Sarah 97963A#2509`).
* The code below is an implementation of the "move to reference pose using an intermediate direction" algorithm described in Chapter 3 of [*Wheeled Mobile Robotics: From Fundamentals Towards Autonomous Systems*](https://www.sciencedirect.com/book/9780128042045/wheeled-mobile-robotics). For more discussions of this algorithm and mobile robot systems in general, please refer to the book.
* Screenshots of the relevant Chapter 3 sections below, for personal use only

<center><img src = "https://drive.google.com/uc?export=view&id=1zDPvxL1-O6ueYUqbed5abrtLpSciZDeB" width = "600"></center>

<center><img src = "https://drive.google.com/uc?export=view&id=1IHCQyya6eh4xGEiSdU3s1c91QiHtBzs5" width = "600"></center>

<center><img src = "https://drive.google.com/uc?export=view&id=1sLAQR1CVbQ5flc3XyEEcAaY5yRmHMYCv" width = "600"></center>

In [ ]:
import numpy as np
from numpy import pi
import matplotlib.pyplot as plt
import math
import matplotlib.animation as animation
from IPython import display

In [ ]:
# initialization ---------------------------------------------------------------
# change the parameters below to test out different conditions
initX, initY = 0, 0
targetX, targetY = 4, 2.5
currentHeading = 135
targetHeading = 225
tolerance = 0.1
Kp_lin = 20
Kp_turn = 1.7

# this determines how long (how many frames) the animation will run. 400 frames takes around 30 seconds to compute.
numOfFrames = 120
dt = 50   # ms, interval between frames
# ------------------------------------------------------------------------------

currentPos = [initX, initY]
targetPos = [targetX, targetY]

def find_min_angle (targetHeading, currentHeading):
  turnAngle = targetHeading - currentHeading
  if turnAngle > 180 or turnAngle < -180 :
    turnAngle = -1 * np.sign(turnAngle) * (360 - abs(turnAngle))
  return turnAngle

def move_to_pose_step (currentPos, currentHeading, targetPos, targetHeading, Kp_lin, Kp_turn, r = 1, turnMax = 100, linMax = 70):
  currentX, currentY = currentPos[0], currentPos[1]
  targetX, targetY = targetPos[0], targetPos[1]
  
  # the angle between the line connecting the robot's current position to the target point and the x axis
  absTargetAngle = math.atan2 ((targetY-currentY), (targetX-currentX)) *180/pi

  # keep absTargetAngle between 0 and 360
  if absTargetAngle < 0:
    absTargetAngle += 360

  # naming convention consistent with the book
  D = math.sqrt((targetX-currentX)**2 + (targetY-currentY)**2)
  alpha = find_min_angle(absTargetAngle, targetHeading)
  errorTerm1 = find_min_angle(absTargetAngle, currentHeading)

  beta = math.atan(r/D) *180/pi

  if alpha < 0:
    beta = -beta

  if abs(alpha) < abs(beta):
    turnError = errorTerm1 + alpha
  else:
    turnError = errorTerm1 + beta

  # apply proportional control to find linear and turn velocities
  linearVel = Kp_lin * D
  turnVel = Kp_turn * turnError

  # when close enough to the target
  closeToTarget = False
  if D < tolerance:
    closeToTarget = True
  if closeToTarget:
    # apply equation 3.13 from Wheeled Mobile Robotics: From Fundamentals Towards Autonomous Systems
    linearVel =  Kp_lin * D * np.sign(math.cos(turnError *pi/180))  # in pct
    # when close enough to the target, overwrite the turn error output from intermediate direction calculations to prevent oscillations
    turnError = find_min_angle(targetHeading, currentHeading)
    turnVel = Kp_turn * math.atan(math.tan(turnError *pi/180)) *180/pi

  # cap the velocities
  if np.abs(linearVel) > linMax:
    linearVel = np.sign(linearVel) * linMax
  if np.abs(turnVel) > turnMax:
    turnVel = np.sign(turnVel) * turnMax

  # prioritize turning
  if linearVel > (100 - np.abs(turnVel)):
    linearVel = 100 - np.abs(turnVel)
  # # don't prioritize turning
  # if linearVel > 100:
  #   linearVel = 100

  return linearVel, turnVel

# the code below is for animation
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------
fig = plt.figure()
trajectory_lines = plt.plot([], '--', color='black')
trajectory_line = trajectory_lines[0]
heading_lines = plt.plot([], '-', color='red')
heading_line = heading_lines[0]
poses = plt.plot([], 'o', color='orange', markersize=10)
pose = poses[0]
# draw a rectangle
rect_lines_1 = plt.plot([], '-', color='orange')
rect_lines_2 = plt.plot([], '-', color='orange')
rect_lines_3 = plt.plot([], '-', color='orange')
rect_lines_4 = plt.plot([], '-', color='orange')
rect_line_1 = rect_lines_1[0]
rect_line_2 = rect_lines_2[0]
rect_line_3 = rect_lines_3[0]
rect_line_4 = rect_lines_4[0]

# other setup, stationary stuff for example
plt.plot([initX, targetX], [initY, targetY], 'x',color='red',markersize=10)
plt.axis("scaled")
plt.xlim (min([initX, targetX])-1, max([initX, targetX])+1)
plt.ylim (min([initY, targetY])-1, max([initY, targetY])+1)

xs = [currentPos[0]]
ys = [currentPos[1]]

f = 0

def draw_square (length, center, orientation):
  global rect_line_1, rect_line_2, rect_line_3, rect_line_4
  corner1 = [center[0] + length/np.sqrt(2)*np.cos((orientation + 45) *pi/180), center[1] + length/np.sqrt(2)*np.sin((orientation + 45) *pi/180)]
  corner2 = [center[0] + length/np.sqrt(2)*np.cos((orientation + 135) *pi/180), center[1] + length/np.sqrt(2)*np.sin((orientation + 135) *pi/180)]
  corner3 = [center[0] + length/np.sqrt(2)*np.cos((orientation - 135) *pi/180), center[1] + length/np.sqrt(2)*np.sin((orientation - 135) *pi/180)]
  corner4 = [center[0] + length/np.sqrt(2)*np.cos((orientation - 45) *pi/180), center[1] + length/np.sqrt(2)*np.sin((orientation - 45) *pi/180)]
  rect_line_1.set_data([corner1[0], corner2[0]], [corner1[1], corner2[1]])
  rect_line_2.set_data([corner2[0], corner3[0]], [corner2[1], corner3[1]])
  rect_line_3.set_data([corner3[0], corner4[0]], [corner3[1], corner4[1]])
  rect_line_4.set_data([corner4[0], corner1[0]], [corner4[1], corner1[1]])

def robot_animation (frame) :
  # define globals
  global currentPos, currentHeading, f

  # call the step function to get info
  # # x drive
  # xVel, yVel, turnVel = move_to_pose_step (currentPos, currentHeading, targetPos, targetHeading, Kp_lin, Kp_turn)
  # tank drive
  linearVel, turnVel = move_to_pose_step (currentPos, currentHeading, targetPos, targetHeading, Kp_lin, Kp_turn)

  if f < 10:
    # xVel, yVel, turnVel = 0, 0, 0
    linearVel, turnVel = 0, 0

  # model: 200rpm drive with 18" width
  #               rpm   /s  circ   feet
  maxLinVelfeet = 200 / 60 * pi*4 / 12
  #               rpm   /s  center angle   deg
  maxTurnVelDeg = 200 / 60 * pi*4 / 9 *180/pi

  # # position update for x drive -------------------------------------------------------------------------------------------------
  # leftFrontVel = xVel - yVel - turnVel
  # leftBackVel = xVel + yVel - turnVel
  # rightFrontVel = xVel + yVel + turnVel
  # rightBackVel = xVel - yVel + turnVel

  # if np.abs(leftFrontVel) > 100 : leftFrontVel = np.sign(leftFrontVel)*100
  # if np.abs(leftBackVel) > 100 : leftBackVel = np.sign(leftBackVel)*100
  # if np.abs(rightFrontVel) > 100 : rightFrontVel = np.sign(rightFrontVel)*100
  # if np.abs(rightBackVel) > 100 : rightBackVel = np.sign(rightBackVel)*100

  # # total x and y vel in local frame
  # xVelLocal = (leftFrontVel/np.sqrt(2) + leftBackVel/np.sqrt(2) + rightFrontVel/np.sqrt(2) + rightBackVel/np.sqrt(2)) / 4.0
  # yVelLocal = (- leftFrontVel/np.sqrt(2) + leftBackVel/np.sqrt(2) + rightFrontVel/np.sqrt(2) - rightBackVel/np.sqrt(2)) / 4.0

  # # total x and y vel in global frame
  # xVelGlobal = np.cos(currentHeading *pi/180) * xVelLocal - np.sin(currentHeading *pi/180) * yVelLocal
  # yVelGlobal = np.sin(currentHeading *pi/180) * xVelLocal + np.cos(currentHeading *pi/180) * yVelLocal

  # linearVel = np.sqrt(xVelGlobal**2 + yVelGlobal**2)

  # # update x and y
  # stepDis_x = xVelGlobal/100 * maxLinVelfeet * dt/1000
  # stepDis_y = yVelGlobal/100 * maxLinVelfeet * dt/1000
  # currentPos[0] += stepDis_x
  # currentPos[1] += stepDis_y

  # # update heading
  # currentHeading += (rightBackVel - leftFrontVel)/2/100 * maxTurnVelDeg * dt/1000
  # # -------------------------------------------------------------------------------------------------------------------------------

  # position update for tank drive ------------------------------------------------------------------------------------------------
  leftSideVel = linearVel - turnVel
  rightSideVel = linearVel + turnVel
  if np.abs(leftSideVel) > 100 : np.sign(leftSideVel)*leftSideVel
  if np.abs(rightSideVel) > 100 : np.sign(rightSideVel)*rightSideVel
  stepDis = (leftSideVel + rightSideVel)/100 * maxLinVelfeet * dt/1000

  # update x and y
  currentPos[0] += stepDis * np.cos(currentHeading*pi/180)
  currentPos[1] += stepDis * np.sin(currentHeading*pi/180)

  # update heading
  currentHeading += (rightSideVel - leftSideVel)/2/100 * maxTurnVelDeg * dt/1000
  # --------------------------------------------------------------------------------------------------------------------------------

  currentHeading = currentHeading%360
  if currentHeading < 0: currentHeading += 360

  # rest of the animation code
  xs.append(currentPos[0])
  ys.append(currentPos[1])

  # draw robot, heading line, pose, and trajectory
  draw_square(0.75, currentPos, currentHeading)
  heading_line.set_data ([currentPos[0], currentPos[0] + 0.75*np.cos(currentHeading/180*pi)], [currentPos[1], currentPos[1] + 0.75*np.sin(currentHeading/180*pi)])
  pose.set_data ((currentPos[0], currentPos[1]))
  trajectory_line.set_data (xs, ys)

  f += 1

anim = animation.FuncAnimation (fig, robot_animation, frames = numOfFrames, interval = dt)
video = anim.to_html5_video()
html = display.HTML (video)
display.display(html)
plt.close()